In [131]:
import os
import json
import ast
import math
import pandas as pd
import gzip
import pickle as pkl
from collections import defaultdict
from datetime import datetime

In [120]:
os.getcwd()

'e:\\Workspaces\\SASRec.pytorch\\notebooks'

In [121]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "data", "Steam")

In [122]:
ROOT_DIR

'e:\\Workspaces\\SASRec.pytorch\\notebooks\\..\\data\\Steam'

In [123]:
# def parse(g):
#     for l in g:
#         yield eval(l)


# countU = defaultdict(lambda: 0)
# countP = defaultdict(lambda: 0)
# line = 0

# dataset_name = "Steam"
# json_file_path = os.path.join(ROOT_DIR, f"reviews_{dataset_name}.json.gz")

# with gzip.open(json_file_path, "r") as g:
#     line = 0
#     for l in parse(g):
#         line += 1
#         if line % 100000 == 0:
#             print(line)

#         product_id = int(l["product_id"])
#         username = l["username"]
#         time = datetime.strptime(l["date"], "%Y-%m-%d")
#         countU[username] += 1
#         countP[product_id] += 1

In [124]:
# keys = set(countP.keys())
# ids = set(df['id'].unique())

In [125]:
# df[df['id']==340]

In [126]:
# keys - ids

In [127]:
with open(os.path.join(ROOT_DIR, "steam_games.json")) as json_file:
    lines = json_file.read().splitlines()
records = [ast.literal_eval(line) for line in lines]

raw_df = pd.DataFrame(records)

In [128]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [154]:
df = raw_df[['id', 'sentiment', 'genres', 'specs', 'price', 'early_access']]

In [155]:
#fix id
df.dropna(subset = ['id'], inplace=True)
df['id'] = df['id'].astype(int)

itemmap = {}
with open(os.path.join(ROOT_DIR, "itemmap.pkl"), 'rb') as f:
    itemmap = pkl.load(f)
df['id'] = df['id'].apply(lambda x: itemmap[x] if x in  itemmap else None)
df.dropna(subset = ['id'], inplace=True)
print(len(df))

13048
<ipython-input-155-97ddd31a8a19>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset = ['id'], inplace=True)
<ipython-input-155-97ddd31a8a19>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(int)
<ipython-input-155-97ddd31a8a19>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [156]:
#fix sentiment
sentiments = ['Unknown', 'Overwhelmingly Negative', "Very Negative", "Negative", "Mostly Negative", "Mixed", "Mostly Positive", "Positive" , "Very Positive" ,"Overwhelmingly Positive"]
sentiments_dict = {
    'Unknown': -1,
    'Overwhelmingly Negative': 0,
    'Very Negative': 1,
    'Negative': 2,
    'Mostly Negative': 3,
    'Mixed': 4,
    'Mostly Positive': 5,
    'Positive': 6,
    'Very Positive': 7,
    'Overwhelmingly Positive': 8,
}
df['sentiment'] = df['sentiment'].fillna('Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: sentiments_dict[x])
df['sentiment'] = df['sentiment'].astype(int)

<ipython-input-156-261ac1949ad6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].fillna('Unknown')
<ipython-input-156-261ac1949ad6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
<ipython-input-156-261ac1949ad6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [157]:
#fix genres
genres = set()
for item in list(raw_df["genres"].dropna()):        
    genres = genres | set(item)
print(len(genres))

def is_in_genere(genre, x):
    try:
        return 1 if genre in x else 0
    except:
        return 0

for genre in genres:
    df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))

df.drop(columns=['genres'],inplace=True)

22
<ipython-input-157-d9d1c6b5298e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))
e:\Workspaces\SASRec.pytorch\.venv_sas_rec\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [158]:
#fix specs
specs = set()
for item in list(raw_df["specs"].dropna()):        
    specs = specs | set(item)
print(len(specs))

def is_in_spec(spec, x):
    try:
        return 1 if spec in x else 0
    except:
        return 0

for spec in specs:
    df[f'spec_{spec}'] = df['specs'].apply(lambda x: is_in_spec(spec, x))

df.drop(columns=['specs'],inplace=True)

40
<ipython-input-158-c31250a17a95>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'spec_{spec}'] = df['specs'].apply(lambda x: is_in_spec(spec, x))


In [159]:
df

,id,sentiment,price,early_access,genre_Web Publishing,genre_Accounting,genre_Photo Editing,genre_Video Production,genre_Sports,genre_Early Access,...,spec_Co-op,spec_Cross-Platform Multiplayer,spec_Keyboard / Mouse,spec_Commentary available,spec_Tracked Motion Controllers,spec_Standing,spec_Full controller support,spec_MMO,spec_In-App Purchases,spec_Game demo
1,3061.0,5,Free To Play,False,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,2019.0,5,Free to Play,False,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,10834.0,4,3.99,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,6827.0,7,9.99,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,3583.0,8,9.99,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32116,6392.0,8,14.99,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32117,2950.0,7,9.99,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32124,10483.0,-1,0.99,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32129,9334.0,-1,1.99,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
#fix price
# strange_prices = ['Free to Play', 'nan','Free','Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']
free_prices = ['Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use', 'Free To Play	', "'Free To Play'", 'Free To Play']
free_prices = free_prices + [df.iloc[1]['price']]
df['price'] = df['price'].apply(lambda x: 0 if x in free_prices else x)
df['price'] = df['price'].apply(lambda x: 0 if x == 'Third-party' else x)
df['price'] = df['price'].apply(lambda x: 499.0 if x == 'Starting at $499.00' else x)
df['price'] = df['price'].apply(lambda x: 449.0 if x == 'Starting at $449.00' else x)
df['price'] = df['price'].fillna(-1)
# df['price'].iloc[0] = 0
# df.dropna(subset = ['price'], inplace=True)

# df['price']=df['price'].apply(lambda x:round(x,2))

<ipython-input-160-21546650a4c6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].apply(lambda x: 0 if x in free_prices else x)
<ipython-input-160-21546650a4c6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].apply(lambda x: 0 if x == 'Third-party' else x)
<ipython-input-160-21546650a4c6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [161]:
df[df['price'] == "Free To Play"]

,id,sentiment,price,early_access,genre_Web Publishing,genre_Accounting,genre_Photo Editing,genre_Video Production,genre_Sports,genre_Early Access,...,spec_Co-op,spec_Cross-Platform Multiplayer,spec_Keyboard / Mouse,spec_Commentary available,spec_Tracked Motion Controllers,spec_Standing,spec_Full controller support,spec_MMO,spec_In-App Purchases,spec_Game demo


In [162]:
# fix early_access
df['early_access'] = df['early_access'].apply(lambda x: 1 if x else 0)

<ipython-input-162-08b71fb7b30e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['early_access'] = df['early_access'].apply(lambda x: 1 if x else 0)


In [163]:
df[df.isnull().any(axis=1)]

,id,sentiment,price,early_access,genre_Web Publishing,genre_Accounting,genre_Photo Editing,genre_Video Production,genre_Sports,genre_Early Access,...,spec_Co-op,spec_Cross-Platform Multiplayer,spec_Keyboard / Mouse,spec_Commentary available,spec_Tracked Motion Controllers,spec_Standing,spec_Full controller support,spec_MMO,spec_In-App Purchases,spec_Game demo


In [164]:
df.to_csv(os.path.join(ROOT_DIR, "items_info.csv"), index=False)

In [ ]:
x = df.set_index('id').T.to_dict('list')

In [ ]:
x[215243]

In [ ]:
from sklearn.decomposition import PCA

pca_df = df.drop(columns=['id'])
pca = PCA(n_components=10)
pca_np = pca.fit_transform(pca_df)
pca_df = pd.DataFrame(pca_np)
pca_df['id'] = df['id']
pca_df = pca_df[['id' ,0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
pca_df.to_csv(os.path.join(ROOT_DIR, "items_info_pca_10.csv"), index=False)

In [ ]:
from sklearn.cluster import FeatureAgglomeration

fagg_df = df.drop(columns=['id'])
fagg = FeatureAgglomeration(n_clusters=10)
fagg_np = fagg.fit_transform(fagg_df)
fagg_df = pd.DataFrame(fagg_np)
fagg_df['id'] = df['id']
fagg_df = fagg_df[['id' ,0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
fagg_df.to_csv(os.path.join(ROOT_DIR, "items_info_fagg_10.csv"), index=False)